# A Neural Network Based Approach to Medical Image Recognition
### Jack Savage, Kyle Burack, Tyler Seppala, Josh Michelberg

---
**Introduction**:

Many communities around the world have little to no access to modern health care. Within our own country, there are two major groups with limited access: those without money for health-care, and those who are far from their nearest healthcare provider. In both circumstances listed, artificial intelligence provides a solution. In poverty-stricken areas, human doctors are costly while an open-source software-based diagnosis system would be cheap. In rural areas without access to experienced or well-trained doctors, medical technicians would be able to perform at a similar level.

![alt text](doctor_shortages.jpg "Doctor Shortages")
**Relevent graphic showcasing shortages of doctors within the United States**

_Further information on the 'HPSA score' metric used in the graph can be found [here](https://bhw.hrsa.gov/shortage-designation/hpsas)_

- **Talk about previous approaches to medical image recognition (CITE STUFF HERE)**

- **Talk about how versatile neural networks are due to transfer learning and training process as a whole (CITE STUFF HERE)**

In this project, we will evaluate the use of convolutional neural networks (CNNs) in classyfing disease from medical imaging. Last semester, we performed the research necessary to sufficiently understand neural networks. This semester's goal is to create a functional model **CHANGE THIS with accuracy equal to that of human doctors CHANGE THIS**.

**Past Work**:

Our first objective for this project was closing the knowledge gap. When we started working last semester, we knew very little about our topic. We built a conceptual and technical understanding of the material, allowing us to focus on implementation this semester. **NEEDS TO BE FLESHED OUT**


- Discuss this semester's initial model (BRIEF)
    - What is a convolutional network (CITE STUFF HERE)
    - What went wrong
        - Failings to account for dataset
        - Using xception default input size of 299x299
- Discuss attempt with retinaNet (EXTENSIVE)
    - What is object detection and retinaNet (CITE STUFF HERE)
    - How does this approach differ from the "vanilla networks" we've been using
    - Talk about repo we used
    - View results of retinaNet model

## This Semester's Work

**Intro:**
- Discuss ensembling methods
    - Discuss subnetworks
- Dataset
    - Discuss dimensionality issues
    - Discuss discrepencies in classes
- Preprocessing of data
    - Discuss techniques such as over/under-sampling

**Results**:
- Tensorboard graphs (BOTH MODELS)
    - Interpretations
- Evaluate network on test imaging 
    - Discuss how discrepencies in dataset could account for results

#### Xception training accuracy
![](training_accuracy_xception_v2.png)
#### Xception training loss
![](training_loss_xception_v2.png)

**Improvements for Next Semester:**
- More ensembled models
- Evaluate new methods from literature
- Evaluate current infrastructure on new dataset

## Code Snippits to Write

**Evaluate trained Network on Test Data**
- Load model
- Run model on test data; save history
- Display metrics in matplotlib

**Draw box around pneumonia**
- Rework old matplotlib solution

**Ensemble method with multiple networks**:
- Figure out keras API to manipulate multiple networks
- Take outputs, average them

In [1]:
'''Using ensembled models to make predictions'''

'Using ensembled models to make predictions'

In [2]:
import cv2
import keras
import matplotlib
import pandas as pd
import numpy as np
import keras.layers as layers
from keras.models import Model
from keras.models import load_model
from keras.layers import Average, Input
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
models = [load_model('xception15.hdf5'),
          load_model('xception15.hdf5'),]

In [16]:
data_dir = r'C:\Users\jecks\Documents\School\GCI\gci_data\test'

datagen = ImageDataGenerator()
datagen = datagen.flow_from_directory(data_dir,
                                     target_size=(1024,1024))

Found 208 images belonging to 2 classes.


### Testing model

In [ ]:
model1 = models[0]
model1.evaluate_generator(datagen)

### Ensembling Models

In [5]:
def ensemble(models, model_input):
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

In [9]:
def ensembleModels(models, model_input):
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models] 
    # averaging outputs
    yAvg=layers.average(yModels) 
    # build model from same input and avg output
    modelEns = Model(inputs=model_input, outputs=yAvg,    name='ensemble')  
   
    return modelEns

In [10]:
model_input = Input(shape=models[0].input_shape[1:]) 
modelEns = ensembleModels(models, model_input)
model.summary()

AttributeError: 'Model' object has no attribute 'model_input'

In [ ]:
theWholeSquad = ensemble(models, models[0].input_shape)

## Resources
- https://www.ruralhealthinfo.org/topics/healthcare-access